In [1]:
import pandas as analytics
import numpy as maths
import math
from numba import cuda , jit
import warnings 
warnings.filterwarnings("ignore")

In [2]:
df_data = analytics.read_csv('data1.csv',header = None , names = ['x0','x1','y'])
df_data

,x0,x1,y
0,22.930,-20.9020,1
1,12.410,2.0033,1
2,-20.686,-33.3030,1
3,46.974,-13.5550,1
4,41.965,20.3680,1
...,...,...,...
195,-29.232,28.4410,-1
196,-10.159,7.3065,-1
197,-32.026,14.9380,-1
198,-49.533,-11.1280,-1


In [37]:
class neural_network :

    def __init__(self,df_data , number_of_hidden_layers = 1, number_of_neurons = [2,1], epsilon = 5e-2, alpha = 0.2) :
        self.number_of_hidden_layers = number_of_hidden_layers
        self.number_of_neurons = number_of_neurons
        self.df_data = df_data
        self.epsilon = epsilon 
        self.N = df_data.shape[0]
        self.alpha = alpha


    def relu(self,x) :
        """Hidden Activation Function"""
        return max(0,x)
    
    def sigmoid(self,w,x,b):
        """Output Function"""
        return 1 / (1 + math.exp(-(w.T @ x + b)))

    def initialisation(self):
        weights = []
        bias = []
        for l in range(self.number_of_hidden_layers + 1):
            if l == 0 : previous_layer = df_data.shape[1] - 1
            else : previous_layer = number_of_neurons[l-1]
            present_layer = number_of_neurons[l]
            weights.append(maths.random.random((previous_layer,present_layer)))
            bias.append(maths.random.random(present_layer).reshape(-1,1))
        self.weights = weights
        self.bias = bias


    def forward_propagation(self,x):
        activations = []
        activations.append(x)   # inputdata point is the initial activation
        
        hidden_layers = []
        
        for l in range(number_of_hidden_layers):
            neurons = self.weights[l] @ activations[l] + self.bias[l]   # it actually should be activations[l+1] but because indexing in python starts from 0, so it is activations[l]
            activation = maths.matrix([self.relu(float(neuron)) for neuron in neurons]).reshape(-1,1)
            hidden_layers.append(neurons)
            activations.append(activation)
        activations.append(sigmoid(weights[-1],activations[-1], bias[-1]))
        
        self.hidden_layers = hidden_layers
        self.activations = activations


    def backward_propagation(self) :
      
        deltas = []
        delta = maths.matrix(maths.multiply(self.activations[-1],  self.sigmoid(self.weights[-1],self.activations[-2], self.bias[-1]) * ( 1- self.sigmoid(self.weights[-1],self.activations[-2], self.bias[-1])))).reshape(-1,1)
        deltas.append(delta)
        
        grad_weights = []
        grad_biases = []
        for l in range(self.number_of_hidden_layers , -1, -1 ): # it should be -1 only. Some errors in activations is not letting it run.
            grad_weight = deltas[0] @ self.activations[l].T
            grad_bias = deltas[0]
            delta = maths.multiply((deltas[0] @ self.weights[1].T).T , maths.matrix([self.relu(float(i)) for i in self.hidden_layers[l-1]]).reshape(-1,1))
            deltas.insert(0,delta)
            grad_weights.append(grad_weight)
            grad_biases.append(grad_bias)
    
        return grad_weights, grad_biases


    def has_converged(self, prev_weights, prev_bias) :
        self._converged = (maths.linalg.norm(maths.matrix(self.weights[0]) - maths.matrix(prev_weights[0])) < self.epsilon) and (maths.linalg.norm(maths.matrix(self.bias[0]) - maths.matrix(prev_bias[0])) < self.epsilon)


    def update_weights(self):
        
        prev_weights = [w + 1 for w in self.weights]
        prev_bias = [b + 1 for b in self.bias]
        
        self._converged = False
        
        while not self._converged:
            prev_weights = weights.copy()
            prev_bias = bias.copy()
            
            for m in range(self.N) :
                x = maths.matrix(self.df_data.iloc[m][:-1]).reshape(-1,1)
                y = self.df_data.iloc[m][-1]
                self.forward_propagation(x)
                grad_weights, grad_bias = self.backward_propagation()
                for l in range(self.number_of_hidden_layers) :
                    self.weights[l] = self.weights[l] - self.alpha/self.N * sum(grad_weights)
                    self.bias[l] = self.bias[l] - self.alpha/self.N * sum(grad_bias)
            
            self.has_converged(prev_weights, prev_bias)
        




    


            

        
        

In [ ]:
ann = neural_network(df_data)
ann.initialisation()
ann.update_weights()


In [3]:
number_of_neurons = [2,1]
number_of_hidden_layers = 1

def relu(x): 
    """Hidden Activation Function"""
    return max(0,x)

def sigmoid(w,x,b):
    """Output Function"""
    return 1 / (1 + math.exp(-(w.T @ x + b)))

In [4]:
weights = []
bias = []
for l in range(number_of_hidden_layers + 1):
    if l == 0 : previous_layer = df_data.shape[1] - 1
    else : previous_layer = number_of_neurons[l-1]
    present_layer = number_of_neurons[l]
    weights.append(maths.random.random((previous_layer,present_layer)))
    bias.append(maths.random.random(present_layer).reshape(-1,1))
    
weights , bias

([array([[0.65513618, 0.48904618],
         [0.25820238, 0.13249368]]),
  array([[0.40336251],
         [0.65869754]])],
 [array([[0.60863117],
         [0.88536448]]),
  array([[0.5163981]])])

In [5]:
x = maths.matrix(df_data.iloc[0][:-1]).reshape(2,1)
y = df_data.iloc[0][-1]
x , y

(matrix([[ 22.93 ],
         [-20.902]]),
 1.0)

In [20]:
# @jit(target_backend = 'cuda')
def forward_propagation(x):
    activations = []
    activations.append(x)   # activations has the first input as the input data point
    
    hidden_layers = []
    
    for l in range(number_of_hidden_layers):
        neurons = weights[l] @ activations[l] + bias[l]   # it actually should be activations[l+1] but because indexing in python starts from 0, so it is activations[l]
        activation = maths.matrix([relu(float(neuron)) for neuron in neurons]).reshape(-1,1)
        hidden_layers.append(neurons)
        activations.append(activation)
    activations.append(sigmoid(weights[-1],activations[-1], bias[-1]))
    return hidden_layers, activations

# @jit(target_backend = 'cuda')
def backward_propagation(hidden_layers, activations) :
    deltas = []
    delta = maths.matrix(maths.multiply(activations[-1],  sigmoid(weights[-1],activations[-2], bias[-1]) * ( 1- sigmoid(weights[-1],activations[-2], bias[-1])))).reshape(-1,1)
    deltas.append(delta)
    
    grad_weights = []
    grad_biases = []
    for l in range(number_of_hidden_layers , -1, -1 ): # it should be -1 only. Some errors in activations is not letting it run.
        grad_weight = deltas[0] @ activations[l].T
        grad_bias = deltas[0]
        delta = maths.multiply((deltas[0] @ weights[1].T).T , maths.matrix([relu(float(i)) for i in hidden_layers[l-1]]).reshape(-1,1))
        deltas.insert(0,delta)
        grad_weights.append(grad_weight)
        grad_biases.append(grad_bias)

    return grad_weights, grad_biases
    
# @jit(target_backend = 'cuda')
def has_converged(prev_weights, prev_bias, weights, bias) :
    converged = False
    epsilon = 2e-2
    converged = (maths.linalg.norm(maths.matrix(weights[0]) - maths.matrix(prev_weights[0])) < epsilon) and (maths.linalg.norm(maths.matrix(bias[0]) - maths.matrix(prev_bias[0])) < epsilon)
    return converged

# @jit(target_backend = 'cuda')
def update_weights(df_data):
    alpha = 0.2
    N = df_data.shape[0]
    
    prev_weights = [w + 1 for w in weights]
    prev_bias = [b + 1 for b in bias]
    
    converged = False
    
    
    while not converged:
        prev_weights = weights.copy()
        prev_bias = bias.copy()
        
        for m in range(df_data.shape[0]) :
            x = maths.matrix(df_data.iloc[m][:-1]).reshape(-1,1)
            y = df_data.iloc[m][-1]
            hidden_layers , activations = forward_propagation(x)
            grad_weights, grad_bias = backward_propagation(hidden_layers, activations)
            for l in range(number_of_hidden_layers) :
                weights[l] = weights[l] - alpha/N * sum(grad_weights)
                bias[l] = bias[l] - alpha/N * sum(grad_bias)
        
        converged = has_converged(prev_weights, prev_bias, weights, bias)
    
    print(weights)
    print()
    print(bias)
    print()
    print()
    

In [21]:
# @jit(target_backend = 'cuda')
update_weights(df_data)

KeyboardInterrupt: 